In [1]:
import json
import os
import pickle
import milsed
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# SETUP STUFF
OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources/'
pump = pickle.load(open('/home/js7561/dev/milsed/models/resources/pump.pkl', 'rb'))

with open(os.path.join(OUTPUT_PATH, 'index_eval.json'), 'r') as fp:
    eval_idx = json.load(fp)['id']

pumpfolder = '/beegfs/js7561/datasets/dcase2017/task4_official/eval/features_silence/'
duration = 10.0

## FOR MODELS

In [3]:
def run_predict_test(version, use_orig_duration=True, save=True, weak_from_strong=False):
    
    # Compute results
    weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')
    params = json.load(open(os.path.join(OUTPUT_PATH, version, 'params.json'), 'r'))
    model, inputs, outputs = milsed.models.MODELS[params['modelname']](pump, alpha=1.0)
    model.load_weights(weight_path)
    
    df_s_all, df_d_all = milsed.eval.predict_eval(
        OUTPUT_PATH, pump, [model], eval_idx, pumpfolder, duration, version, use_tqdm=True, 
        use_orig_duration=use_orig_duration, save_jams=save, weak_from_strong=weak_from_strong,
        using_test_set=False)
        
    return df_s_all, df_d_all

### RUN BEST MODELS ON EVAL SET

In [4]:
models = ['model011', 'model024', 'model026bm']

for version in models:
    _ = run_predict_test(version, use_orig_duration=True, save=True, weak_from_strong=False)
    _ = run_predict_test(version, use_orig_duration=True, save=True, weak_from_strong=True)

Evaluating the model: 100%|██████████| 1103/1103 [01:51<00:00, 10.25it/s]


In [7]:
models = ['model007', 'model015', 'model025bm']

for version in models:
    _ = run_predict_test(version, use_orig_duration=True, save=True, weak_from_strong=False)
    _ = run_predict_test(version, use_orig_duration=True, save=True, weak_from_strong=True)

Evaluating the model: 100%|██████████| 1103/1103 [01:25<00:00, 13.11it/s]


## FOR ENSEMBLES

In [5]:
def run_predict_test_ensemble(ensemble_version, model_versions, use_orig_duration=True, save=True, 
                              weak_from_strong=False):
    
    # Compute results
    models = []
    for version in model_versions:
        weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')
        params = json.load(open(os.path.join(OUTPUT_PATH, version, 'params.json'), 'r'))
        model, inputs, outputs = milsed.models.MODELS[params['modelname']](pump, alpha=1.0)
        model.load_weights(weight_path)
        models.append(model)
    
    df_s_all, df_d_all = milsed.eval.predict_eval(
        OUTPUT_PATH, pump, models, eval_idx, pumpfolder, duration, ensemble_version, use_tqdm=True, 
        use_orig_duration=use_orig_duration, save_jams=save, weak_from_strong=weak_from_strong,
        using_test_set=False)
        
    return df_s_all, df_d_all

### RUN BEST ENSEMBLES ON EVAL SET

In [6]:
ensembles = ['ensemble024', 'ensemble047']
model_versions = [['model007', 'model024'], ['model007', 'model011', 'model015']]

for ensemble_version, mods in zip(ensembles, model_versions):
    _ = run_predict_test_ensemble(ensemble_version, mods, use_orig_duration=True, save=True, weak_from_strong=False)
    _ = run_predict_test_ensemble(ensemble_version, mods, use_orig_duration=True, save=True, weak_from_strong=True)

Evaluating the model: 100%|██████████| 1103/1103 [03:24<00:00,  5.42it/s]


## DEBUG STUFF

In [8]:
df_s_all, df_d_all = run_predict_test('model012', use_orig_duration=True, save=True, weak_from_strong=False)

Evaluating the model: 100%|██████████| 1103/1103 [01:27<00:00, 12.59it/s]


In [21]:
df_s_all

,filename,start_time,end_time,label
0,audio/Y-3exNVlj92w_30.000_40.000.wav,0.0,10.000000,Motorcycle
0,audio/YA437a4Y_xag_230.000_240.000.wav,0.0,10.000000,Car
0,audio/Y-3gSkrDKNSA_27.000_37.000.wav,0.0,10.000000,Truck
0,audio/Y-1X7kpLnOpM_60.000_70.000.wav,0.0,10.000000,Train
0,audio/Y0rVBXpbgO8s_30.000_40.000.wav,0.0,10.000000,Screaming
0,audio/YA-GNszKtjJc_93.000_103.000.wav,0.0,10.000000,"Air horn, truck horn"
0,audio/Y-BGebo8V4XY_30.000_40.000.wav,0.0,10.000000,Car
0,audio/Y0QKet-tdquc_30.000_40.000.wav,0.0,10.000000,Reversing beeps
1,audio/Y0QKet-tdquc_30.000_40.000.wav,0.0,10.000000,Truck
0,audio/Y06RreMb5qbE_0.000_10.000.wav,0.0,10.000000,Ambulance (siren)


In [22]:
eval_idx

['Y-3exNVlj92w_30.000_40.000',
 'YA437a4Y_xag_230.000_240.000',
 'Y-VULyMtKazE_0.000_7.000',
 'Y-3gSkrDKNSA_27.000_37.000',
 'Y-1X7kpLnOpM_60.000_70.000',
 'Y-x70B12Mb-8_30.000_40.000',
 'Y-Em3OpyaefM_30.000_40.000',
 'Y0rVBXpbgO8s_30.000_40.000',
 'YA-GNszKtjJc_93.000_103.000',
 'Y-BGebo8V4XY_30.000_40.000',
 'Y0QKet-tdquc_30.000_40.000',
 'Y-xz75wUCln8_50.000_60.000',
 'Y06RreMb5qbE_0.000_10.000',
 'Y-UCf_-3yzWU_290.000_300.000',
 'Y-XRiLbb3Syo_2.000_12.000',
 'Y-Qfk_Q2ctBs_30.000_40.000',
 'Y--zbPxnl27o_20.000_30.000',
 'Y-r8mfjRiHrU_30.000_40.000',
 'Y-0-jXXldDOU_10.000_20.000',
 'Y-KmDAgYb0Uo_100.000_110.000',
 'Y-PRrNx6_MD0_16.000_26.000',
 'Y9fCibkUT_gQ_30.000_40.000',
 'Y0Ees8KFCUXM_30.000_40.000',
 'Y-10fWp7Pqs4_30.000_40.000',
 'Y-60XojQWWoc_30.000_40.000',
 'Y-hLSc9aPOms_13.000_23.000',
 'Y--ivFZu-hlc_30.000_40.000',
 'Y0HvYkBXQ44A_30.000_40.000',
 'Y-qmfWP_yzn4_30.000_40.000',
 'Y-KlN_AXMM0Q_30.000_40.000',
 'Y-2LJWaL2PuA_30.000_40.000',
 'Y-0I89-H0AFo_26.000_36.000',
 'Y0I

In [23]:
uniquefiles_s = np.unique([v.replace('audio/', '').replace('.wav', '') for v in df_s_all.filename.values])

In [24]:
uniquefiles_s.shape

(488,)

In [15]:
uniquefiles_s

array(['Y---lTs1dxhU_30.000_40.000', 'Y--0w1YA1Hm4_30.000_40.000',
       'Y--330hg-Ocw_30.000_40.000', 'Y--73E04RpiQ_0.000_9.000',
       'Y--8puiAGLhs_30.000_40.000', 'Y--9VR_F7CtY_30.000_40.000',
       'Y--P4wuph3Mc_0.000_8.000', 'Y--QvRbvnbUE_30.000_40.000',
       'Y--Zz7BgxSUg_30.000_40.000', 'Y--iFD6IyQW8_30.000_40.000',
       'Y--ivFZu-hlc_30.000_40.000', 'Y--jGnLqFsQ4_24.000_34.000',
       'Y--jc0NAxK8M_30.000_40.000', 'Y--ljM2Kojag_30.000_40.000',
       'Y--p-rk_HBuU_30.000_40.000', 'Y--v1WjOJv-w_150.000_160.000',
       'Y--wuU7kzB5o_30.000_40.000', 'Y--xDffQ9Mwo_30.000_40.000',
       'Y--yaQA8d1dI_6.000_16.000', 'Y--zLzL0sq3M_30.000_40.000',
       'Y-0-jXXldDOU_10.000_20.000', 'Y-09rxiqNNEs_30.000_40.000',
       'Y-0B-egfXU7E_30.000_40.000', 'Y-0B_CYyG5Dg_30.000_40.000',
       'Y-0CIk-OOp7Y_30.000_40.000', 'Y-0Eem_FuIto_15.000_25.000',
       'Y-0HsrVfb5vc_20.000_30.000', 'Y-0JqTq_4jaE_40.000_50.000',
       'Y-0P6VDQ1YDs_80.000_90.000', 'Y-0PrEsytvc0_30.000_40.000'

In [16]:
len(eval_idx)

488

In [30]:
df_s_all2 = df_s_all

In [32]:
df_s_all2.head()

,filename,start_time,end_time,label
0,audio/Y-3exNVlj92w_30.000_40.000.wav,0.0,10.0,Motorcycle
0,audio/YA437a4Y_xag_230.000_240.000.wav,0.0,10.0,Car
0,audio/Y-3gSkrDKNSA_27.000_37.000.wav,0.0,10.0,Truck
0,audio/Y-1X7kpLnOpM_60.000_70.000.wav,0.0,10.0,Train
0,audio/Y0rVBXpbgO8s_30.000_40.000.wav,0.0,10.0,Screaming


In [33]:
df = pd.DataFrame(columns=['filename', 'start_time', 'end_time', 'label'])
df.loc[-1, 'filename'] = 'test'
df

,filename,start_time,end_time,label
-1,test,NaN,NaN,NaN


In [34]:
added = 0 
for filename in eval_idx:
    if filename not in uniquefiles_s:
        df = pd.DataFrame(columns=['filename', 'start_time', 'end_time', 'label'])
        df.loc[-1, 'filename'] = 'audio/{}.wav'.format(filename)
        df_s_all2 = df_s_all2.append(df)
        added += 1
added

146

In [35]:
df_s_all2

,filename,start_time,end_time,label
0,audio/Y-3exNVlj92w_30.000_40.000.wav,0.0,10.000000,Motorcycle
0,audio/YA437a4Y_xag_230.000_240.000.wav,0.0,10.000000,Car
0,audio/Y-3gSkrDKNSA_27.000_37.000.wav,0.0,10.000000,Truck
0,audio/Y-1X7kpLnOpM_60.000_70.000.wav,0.0,10.000000,Train
0,audio/Y0rVBXpbgO8s_30.000_40.000.wav,0.0,10.000000,Screaming
0,audio/YA-GNszKtjJc_93.000_103.000.wav,0.0,10.000000,"Air horn, truck horn"
0,audio/Y-BGebo8V4XY_30.000_40.000.wav,0.0,10.000000,Car
0,audio/Y0QKet-tdquc_30.000_40.000.wav,0.0,10.000000,Reversing beeps
1,audio/Y0QKet-tdquc_30.000_40.000.wav,0.0,10.000000,Truck
0,audio/Y06RreMb5qbE_0.000_10.000.wav,0.0,10.000000,Ambulance (siren)


In [25]:
df_s_all2.to_csv('~/test_dcase_output.txt', header=False, index=False, sep='\t')

NameError: name 'df_s_all2' is not defined

In [33]:
df_s_all.to_csv('~/test_dcase_output.txt', header=False, index=False, sep='\t')

In [34]:
with open('/home/js7561/test_dcase_output.txt', "a") as myfile:
    for n, x in enumerate(eval_idx):
        if n+1==len(eval_idx):
            myfile.write('{}'.format(x))
        else:
            myfile.write('{}\n'.format(x))        

In [29]:
eval_idx

['Y-3exNVlj92w_30.000_40.000',
 'YA437a4Y_xag_230.000_240.000',
 'Y-VULyMtKazE_0.000_7.000',
 'Y-3gSkrDKNSA_27.000_37.000',
 'Y-1X7kpLnOpM_60.000_70.000',
 'Y-x70B12Mb-8_30.000_40.000',
 'Y-Em3OpyaefM_30.000_40.000',
 'Y0rVBXpbgO8s_30.000_40.000',
 'YA-GNszKtjJc_93.000_103.000',
 'Y-BGebo8V4XY_30.000_40.000',
 'Y0QKet-tdquc_30.000_40.000',
 'Y-xz75wUCln8_50.000_60.000',
 'Y06RreMb5qbE_0.000_10.000',
 'Y-UCf_-3yzWU_290.000_300.000',
 'Y-XRiLbb3Syo_2.000_12.000',
 'Y-Qfk_Q2ctBs_30.000_40.000',
 'Y--zbPxnl27o_20.000_30.000',
 'Y-r8mfjRiHrU_30.000_40.000',
 'Y-0-jXXldDOU_10.000_20.000',
 'Y-KmDAgYb0Uo_100.000_110.000',
 'Y-PRrNx6_MD0_16.000_26.000',
 'Y9fCibkUT_gQ_30.000_40.000',
 'Y0Ees8KFCUXM_30.000_40.000',
 'Y-10fWp7Pqs4_30.000_40.000',
 'Y-60XojQWWoc_30.000_40.000',
 'Y-hLSc9aPOms_13.000_23.000',
 'Y--ivFZu-hlc_30.000_40.000',
 'Y0HvYkBXQ44A_30.000_40.000',
 'Y-qmfWP_yzn4_30.000_40.000',
 'Y-KlN_AXMM0Q_30.000_40.000',
 'Y-2LJWaL2PuA_30.000_40.000',
 'Y-0I89-H0AFo_26.000_36.000',
 'Y0I

## Format still not OK...

In [10]:
def readLabels(filepath):
    try:
        #Filename will act as key and labels list will be the value 
        labelsDict = {}
        with open(filepath) as filename:
            for line in filename:
                lineArr = line.split("\t")
                #audioFile must be present, make it hard
                if len(lineArr) == 4:
                    if float(lineArr[2].strip()) == 0.0:
                        continue
                    else:
                        audioFile = lineArr[0].split(".wav")[0].split(".flac")[0].strip()
                else:
                    audioFile = lineArr[0].split(".wav")[0].split(".flac")[0].strip()

                try:
                    startTime = lineArr[1].strip()
                except Exception as ex1:
                    startTime = ""
                try:
                    endTime = lineArr[2].strip()
                except Exception as ex2:
                    endTime = ""
                try:
                    label = lineArr[3].strip()
                except Exception as ex3:
                    label = ""

                if audioFile not in labelsDict.keys():
                    #does not exist
                    if label is not "":
                        labelsDict[audioFile] = [label]
                    else:
                        labelsDict[audioFile] = []
                else:
                    #exists
                    if label is not "":
                        labelsDict[audioFile].append(label)

        filename.close()
        #Debug Print
        #for key in self.labelsDict.keys():
        #       print str(key) + ":" + str(self.labelsDict[key])

    except Exception as ex:
        print("Fileformat of the file " + str(filepath) + " is invalid.")
        raise ex

In [11]:
filepath = '/home/js7561/dev/milsed/models/resources/model011/predictions_eval/pred_test_weak.txt'

In [12]:
readLabels(filepath)

Fileformat of the file /home/js7561/dev/milsed/models/resources/model011/predictions_eval/pred_test_weak.txt is invalid.


ValueError: could not convert string to float: 

In [13]:
line = 'audio/Y-P_XDJt4p_s_30.000_40.000.wav			'

In [14]:
line

'audio/Y-P_XDJt4p_s_30.000_40.000.wav\t\t\t'

In [15]:
lineArr = line.split("\t")

In [16]:
lineArr

['audio/Y-P_XDJt4p_s_30.000_40.000.wav', '', '', '']

In [17]:
len(lineArr)

4

In [18]:
lineArr[2].strip()

''

In [14]:
added_d = 0
uniquefiles_d = np.unique(
    ['Y'+v.replace('.wav', '') for v in
     df_d_all.filename.values])

missing_files_d = []
for filename in eval_idx:
    if filename not in uniquefiles_d:
        # df = pd.DataFrame(
        #     columns=['filename', 'start_time', 'end_time', 'label'])
        # df.loc[-1, 'filename'] = 'audio/{}.wav'.format(filename)
        # df_d_all = df_d_all.append(df)
        missing_files_d.append(filename)
        added_d += 1

In [15]:
len(missing_files_d)

48

In [16]:
missing_files_d

['Y3NX4HaOVBoo_240.000_250.000',
 'Y2FEhG1UXb_E_370.000_380.000',
 'Y8m-a_6wLTkU_230.000_240.000',
 'Y8rzhhvS0tGc_30.000_40.000',
 'Yy4Ko6VNiqB0_30.000_40.000',
 'YWg4ik5zZxBc_250.000_260.000',
 'Y1_hGvbEiYAs_30.000_40.000',
 'Y1BgqrhbyRFw_30.000_40.000',
 'YfNcrlqPrAqM_30.000_40.000',
 'YI-HlrcP6Qg4_30.000_40.000',
 'YehYwty_G2L4_13.000_23.000',
 'YnlYlNF30bVg_30.000_40.000',
 'YGmbNjZi4xBw_30.000_40.000',
 'Y-0RWZT-miFs_420.000_430.000',
 'YG6A-sT2DOjY_30.000_40.000',
 'YAIFvFuZPr68_30.000_40.000',
 'YWLXQgcx8qTI_30.000_40.000',
 'YG--718JDmAQ_0.000_10.000',
 'YY-4dtrP-RNo_7.000_17.000',
 'YFjyZV8zIJ0k_30.000_40.000',
 'Y0lpPdWvg7Eo_0.000_10.000',
 'YLSZPNwZex9s_30.000_40.000',
 'Ycs-RPPsg_ks_30.000_40.000',
 'Y8kbHA22EWd0_330.000_340.000',
 'Y-2sE5CH8Wb8_30.000_40.000',
 'YAyfuBDN3Vdw_40.000_50.000',
 'Y84flVacRHUI_21.000_31.000',
 'Y38F6eeIR-s0_30.000_40.000',
 'YOEN0TySl1Jw_10.000_20.000',
 'Y7eeN-fXbso8_20.000_30.000',
 'YBs2KqqI9F_k_30.000_40.000',
 'YAedlWfHafgw_21.000_31.000',

In [13]:
df_d_all

,filename,start_time,end_time,label
0,eyFPHlybqDg_30.000_40.000.wav,0.000000,10.000000,Car
0,xuMBy2NoROI_30.000_40.000.wav,0.000000,0.046440,Car
1,xuMBy2NoROI_30.000_40.000.wav,3.366893,10.000000,Car
0,ikmE_kRvDAc_30.000_40.000.wav,2.995374,4.202812,Motorcycle
1,ikmE_kRvDAc_30.000_40.000.wav,8.753923,10.000000,Car
0,cB1jkzgH2es_150.000_160.000.wav,2.716735,4.318912,Car
1,cB1jkzgH2es_150.000_160.000.wav,8.219864,8.475283,Car
2,cB1jkzgH2es_150.000_160.000.wav,8.475283,10.000000,Bus
3,cB1jkzgH2es_150.000_160.000.wav,8.475283,10.000000,Car
0,aOoZ0bCoaZw_30.000_40.000.wav,1.671837,10.000000,Skateboard
